In [17]:
import pandas as pd

train_off = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
train_off = train_off.dropna(subset = ['Coupon_id'])

In [18]:
import datetime
 
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        if days.days > 15:
            return 0
        else:
            return 1
    else:
        return 0
        
train_off['less_15'] = train_off.apply(days, axis = 1)

In [19]:
train_off.Distance = train_off.Distance.fillna(1)
test.Distance = test.Distance.fillna(1)

In [20]:
train = train_off[['User_id', 'less_15']]
a = train.less_15.groupby(train['User_id']).sum()
a = a.reset_index()
train = a
train_off = train_off.drop(columns = 'less_15')

In [21]:
train_off = pd.merge(train_off, train, on = 'User_id', how = 'left')
test = pd.merge(test, train, on = 'User_id', how = 'left')

In [22]:
test = test.fillna(0)

In [23]:
train_off["Week"] = train_off["Date_received"]
train_off.Week = train_off.Week.astype("int")
train_off.Week = train_off.Week.astype("str")
train_off["Week"] = pd.to_datetime(train_off["Week"]).apply(lambda x : x.weekday())

In [24]:
test["Week"] = test["Date_received"]
test.Week = test.Week.astype("int")
test.Week = test.Week.astype("str")
test["Week"] = pd.to_datetime(test["Week"]).apply(lambda x : x.weekday())

In [25]:
train_off['result'] = train_off[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

In [26]:
X_train = train_off[['Discount_rate', 'Distance', 'less_15', 'Week']]
y_train = train_off.result
X_test = test[['Discount_rate', 'Distance', 'less_15', 'Week']]

In [11]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [12]:
train_name = X_train.columns.tolist()
test_name = X_test.columns.tolist()
m = list(set(train_name) - set(test_name))

for i in m:
    X_test[i] = 1
    
X_train['Discount_rate_500:30'] = 1

In [13]:
X_train_name = X_train.columns.tolist()
X_test = X_test[X_train_name]

In [14]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)
predict = clf.predict_proba(X_test)
save = test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

In [15]:
save.to_csv('E:/tianchi/20180718/less_15.csv', index = None)